In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
#!/usr/bin/env python
# coding=utf-8
# Copyright 2021 The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Fine-tuning the library models for masked language modeling (BERT, ALBERT, RoBERTa...)
on a text file or a dataset without using HuggingFace Trainer.
Here is the full list of checkpoints on the hub that can be fine-tuned by this script:
https://huggingface.co/models?filter=masked-lm
"""
# You can also adapt this script on your own mlm task. Pointers for this are left as comments.

import argparse
import logging
import math
import os
import random
import sys

import torch
from tqdm.auto import tqdm
from dataclasses import field, dataclass
from typing import Dict, List, Union, Optional

# ----------------------------------- #
#           All Imports
# ----------------------------------- #
import os # generic
import time # logging
from tqdm.auto import tqdm # custom progress bar

import io
import json # load/write data
import torch 
import numpy as np
import pandas as pd

# 🤗 Datasets
import datasets
from datasets import load_dataset #, concatenate_datasets

# s2orc load dataset (not preprocessed)
from utils.s2orc.loader import s2ortc_loader

# 🤗 Tranformers
import transformers
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoTokenizer, 
    AutoModel, 
    AutoModelForMaskedLM,
    AutoConfig,
    PreTrainedTokenizer, 
    DataCollatorForLanguageModeling, 
    Trainer, 
    BertForMaskedLM,
    HfArgumentParser,
    TrainingArguments as HfTrainingArguments,
    SchedulerType,
    get_scheduler,
    set_seed,
)

# Dataset configuration files
from utils.config.datasets import S2orcConfig, KeyPHConfig
from utils.config.execution import RunConfig

# data types
# from torch.utils.data.dataloader import DataLoader
from torch.utils.data import Dataset, DataLoader

# accelerator for speed up experiments/runs
from accelerate import Accelerator
    
logger = logging.getLogger(__name__)
MODEL_CONFIG_CLASSES = list(MODEL_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

### Load dataset wrapper
---

The function `load_dataset_wrapper` is thougth to wrap the original `load_dataset` function from `hf`. This is because we provide 2 additional custom datasets (other than what's inside `huggingface/datasets.hub` repository).
The datasets we provide will have some custom parameters we treat at the beginning.

In [4]:
from utils.general import load_dataset_wrapper

In [5]:
from utils.parsers.args_parser import (
    parse_args
)

In [7]:
def main(args_list):
    # Pass the args_list to parse_args. We will use it for 
    # args = parse_args(args_list)
    
    dataset_args, training_args, s2orc_args, keyph_args, run_args, log_args, embedding_args = parse_args(args_list)
    
    # Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
    accelerator = Accelerator()
    # Make one log on every process with the configuration for debugging.
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )
    logger.info(accelerator.state)

    # Setup logging, we only want one process per machine to log things on the screen.
    # accelerator.is_local_main_process is only True for one process per machine.
    logger.info(f"Accelerator local main process: {accelerator.is_local_main_process}")
    logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
    if accelerator.is_local_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    # If passed along, set the training seed now.
    if training_args.seed is not None:
        set_seed(training_args.seed)

    # Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
    # or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
    # (the dataset will be downloaded automatically from the datasets Hub).
    #
    # For CSV/JSON files, this script will use the column called 'text' or the first column if no column called
    # 'text' is found. You can easily tweak this behavior (see below).
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    if dataset_args.dataset_name is not None:
        # --------------------- #
        #     added  here       #
        # --------------------- #
        custom_load_dataset = load_dataset_wrapper()
        raw_datasets = custom_load_dataset(dataset_args, training_args, s2orc_args, keyph_args, run_args, log_args, embedding_args)
    else:
        data_files = {}
        if dataset_args.train_file is not None:
            data_files["train"] = dataset_args.train_file
        if dataset_args.validation_file is not None:
            data_files["validation"] = dataset_args.validation_file
        extension = dataset_args.train_file.split(".")[-1]
        if extension == "txt":
            extension = "text"
        elif extension == "jsonl": # jsonl files are file with json element per row
            extension = "json"
        raw_datasets = load_dataset(extension, data_files=data_files)
        
    print(raw_datasets)
    # See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
    # https://huggingface.co/docs/datasets/loading_datasets.html.

     def format_columns_names(raw_datasets):
    
        def names_dict_generator(names_tuple):
            names_map = dict()
            for key, values in names_tuple:
                for value in values:
                    names_map[value] = key
            return names_map
                
        names_tuple = [('train', ['train']), ('test', ['test','debug']), ('validation', ['validation','valid'])]
        names_map = names_dict_generator(names_tuple)
        split_names = raw_datasets.keys()
        for split_name in split_names:
            new_split_name = names_map.get(split_name)
            if split_name != new_split_name:
                raw_datasets[new_split_name] = raw_datasets.pop(split_name)
            
        return raw_datasets
    
    logger.info(f"Formatting DatasetDict keys")
    raw_datasets = format_columns_names(raw_datasets)
    

    # Load pretrained model and tokenizer
    #
    # In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.
    if dataset_args.config_name:
        config = AutoConfig.from_pretrained(dataset_args.config_name)
    elif dataset_args.model_name_or_path:
        config = AutoConfig.from_pretrained(dataset_args.model_name_or_path)
    else:
        config = CONFIG_MAPPING[training_args.model_type]()
        logger.warning("You are instantiating a new config instance from scratch.")

    if dataset_args.tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(dataset_args.tokenizer_name, use_fast=not dataset_args.use_slow_tokenizer)
    elif dataset_args.model_name_or_path:
        tokenizer = AutoTokenizer.from_pretrained(dataset_args.model_name_or_path, use_fast=not dataset_args.use_slow_tokenizer)
    else:
        raise ValueError(
            "You are instantiating a new tokenizer from scratch. This is not supported by this script."
            "You can do it from another script, save it, and load it from here, using --tokenizer_name."
        )

    if dataset_args.model_name_or_path:
        model = AutoModelForMaskedLM.from_pretrained(
            dataset_args.model_name_or_path,
            from_tf=bool(".ckpt" in dataset_args.model_name_or_path),
            config=config,
        )
    else:
        logger.info("Training new model from scratch")
        model = AutoModelForMaskedLM.from_config(config)

    model.resize_token_embeddings(len(tokenizer))

    # Preprocessing the datasets.
    # First we tokenize all the texts.
    column_names = raw_datasets["train"].column_names
    text_column_name = "text" if "text" in column_names else column_names[0]

    if training_args.max_seq_length is None:
        max_seq_length = tokenizer.model_max_length
        if max_seq_length > 1024:
            logger.warning(
                f"The tokenizer picked seems to have a very large `model_max_length` ({tokenizer.model_max_length}). "
                "Picking 1024 instead. You can change that default value by passing --max_seq_length xxx."
            )
            max_seq_length = 1024
    else:
        if training_args.max_seq_length > tokenizer.model_max_length:
            logger.warning(
                f"The max_seq_length passed ({training_args.max_seq_length}) is larger than the maximum length for the"
                f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
            )
        max_seq_length = min(training_args.max_seq_length, tokenizer.model_max_length)

    if training_args.line_by_line:
        # When using line_by_line, we just tokenize each nonempty line.
        padding = "max_length" if dataset_args.pad_to_max_length else False

        def tokenize_function(examples):
            # Remove empty lines
            examples["text"] = [line for line in examples["text"] if len(line) > 0 and not line.isspace()]
            return tokenizer(
                examples["text"],
                padding=padding,
                truncation=True,
                max_length=max_seq_length,
                # We use this option because DataCollatorForLanguageModeling (see below) is more efficient when it
                # receives the `special_tokens_mask`.
                return_special_tokens_mask=True,
            )

        tokenized_datasets = raw_datasets.map(
            tokenize_function,
            batched=True,
            num_proc=training_args.preprocessing_num_workers,
            remove_columns=[text_column_name],
            load_from_cache_file=not training_args.overwrite_cache,
        )
    else:
        # Otherwise, we tokenize every text, then concatenate them together before splitting them in smaller parts.
        # We use `return_special_tokens_mask=True` because DataCollatorForLanguageModeling (see below) is more
        # efficient when it receives the `special_tokens_mask`.
        def tokenize_function(examples):
            return tokenizer(examples[text_column_name], return_special_tokens_mask=True)

        tokenized_datasets = raw_datasets.map(
            tokenize_function,
            batched=True,
            num_proc=training_args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not training_args.overwrite_cache,
        )

        # Main data processing function that will concatenate all texts from our dataset and generate chunks of
        # max_seq_length.
        def group_texts(examples):
            # Concatenate all texts.
            concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
            total_length = len(concatenated_examples[list(examples.keys())[0]])
            # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
            # customize this part to your needs.
            total_length = (total_length // max_seq_length) * max_seq_length
            # Split by chunks of max_len.
            result = {
                k: [t[i : i + max_seq_length] for i in range(0, total_length, max_seq_length)]
                for k, t in concatenated_examples.items()
            }
            return result

        # Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a
        # remainder for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value
        # might be slower to preprocess.
        #
        # To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
        # https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map

        tokenized_datasets = tokenized_datasets.map(
            group_texts,
            batched=True,
            num_proc=training_args.preprocessing_num_workers,
            load_from_cache_file=not training_args.overwrite_cache,
        )

    train_dataset = tokenized_datasets["train"]
    eval_dataset = tokenized_datasets["validation"]

    # Log a few random samples from the training set:
    for index in random.sample(range(len(train_dataset)), 3):
        logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

    # Data collator
    # This one will take care of randomly masking the tokens.
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, mlm_probability=training_args.mlm_probability)

    # DataLoaders creation:
    train_dataloader = DataLoader(
        train_dataset, shuffle=True, collate_fn=data_collator, batch_size=training_args.per_device_train_batch_size
    )
    eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=training_args.per_device_eval_batch_size)

    # Optimizer
    # Split weights in two groups, one with weight decay and the other not.
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=training_args.learning_rate)

    # Prepare everything with our `accelerator`.
    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader
    )

    # Note -> the training dataloader needs to be prepared before we grab his length below (cause its length will be
    # shorter in multiprocess)

    # Scheduler and math around the number of training steps.
    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
    if training_args.max_train_steps is None:
        training_args.max_train_steps = training_args.num_train_epochs * num_update_steps_per_epoch
    else:
        training_args.num_train_epochs = math.ceil(training_args.max_train_steps / num_update_steps_per_epoch)

    lr_scheduler = get_scheduler(
        name=training_args.lr_scheduler_type,
        optimizer=optimizer,
        num_warmup_steps=training_args.num_warmup_steps,
        num_training_steps=training_args.max_train_steps,
    )

    # Train!
    total_batch_size = training_args.per_device_train_batch_size * accelerator.num_processes * training_args.gradient_accumulation_steps

    logger.info("***** Running training *****")
    logger.info(f"  Num examples = {len(train_dataset)}")
    logger.info(f"  Num Epochs = {training_args.num_train_epochs}")
    logger.info(f"  Instantaneous batch size per device = {training_args.per_device_train_batch_size}")
    logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
    logger.info(f"  Gradient Accumulation steps = {training_args.gradient_accumulation_steps}")
    logger.info(f"  Total optimization steps = {training_args.max_train_steps}")
    # Only show the progress bar once on each machine.
    progress_bar = tqdm(range(training_args.max_train_steps), disable=not accelerator.is_local_main_process)
    completed_steps = 0

    for epoch in range(training_args.num_train_epochs):
        model.train()
        for step, batch in enumerate(train_dataloader):
            outputs = model(**batch)
            loss = outputs.loss
            loss = loss / training_args.gradient_accumulation_steps
            accelerator.backward(loss)
            if step % training_args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(1)
                completed_steps += 1

            if completed_steps >= training_args.max_train_steps:
                break

        model.eval()
        losses = []
        for step, batch in enumerate(eval_dataloader):
            with torch.no_grad():
                outputs = model(**batch)

            loss = outputs.loss
            losses.append(accelerator.gather(loss.repeat(training_args.per_device_eval_batch_size)))

        losses = torch.cat(losses)
        losses = losses[: len(eval_dataset)]
        perplexity = math.exp(torch.mean(losses))

        logger.info(f"epoch {epoch}: perplexity: {perplexity}")

    if args.output_dir is not None:
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(training_args.output_dir, save_function=accelerator.save)

In [12]:
if __name__ == "__main__":
    args_list = [

        # DatasetArguments
        "--model_name_or_path"           , "allenai/scibert_scivocab_uncased",
        "--dataset_name"                 , "keyphrase", # "s2orc",
        "--dataset_config_name"          , "inspec", # "full",
        
        # TrainingArguments        
            # seed for reproducibility of experiments
        "--seed"                         , '1234', 
        "--output_dir"                   , "output",
        "--debug"                        , 'False',
        
        "--run_name"                     , "scibert-keyph", # "scibert-s2orc",
        "--num_train_epochs"             , '1',
        "--per_device_train_batch_size"  , "32",
        "--per_device_eval_batch_size"   , "32",
            # custom added
        "--max_seq_length"               , '512',
        
        # S2orcArguments & KeyPhArguments
        "--dataset_path"                 , "/home/vivoli/Thesis/data",
        "--data"                         , "abstract",
        "--target"                       , "title",             
        "--classes"                      , "keywords",
        
        # S2orcArguments
        # "--idxs"                         , '0',
        # "--zipped"                       , 'True',
        #     # list
        # "--mag_field_of_study"           , "Computer Science",    
        #     # list
        # "--data"                         , "abstract",
        #     # list
        # "--target"                       , "title",             
        #     # list
        # "--classes"                      , "mag_field_of_study",
        # "--keep_none_papers"             , 'False',
        # "--keep_unused_columns"          , 'False',
        
        # RunArguments
        # "--run_name"                     , "scibert-s2orc",
        "--run_number"                   , '0',
        "--run_iteration"                , '0',
        
        # LoggingArguments
        "--verbose"                      , 'True',
        # "--debug"                        , False,
        "--time"                         , 'False',
        "--callback"                     , "WandbCallback",
        
        # EmbeddingArguments
        # "--max_seq_length"               , '512',
        # "--pooling"                      , 'none',
        # "--batch_size"                   , '32'
    ]
    
    main(args_list)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
05/05/2021 15:40:56 - INFO - __main__ -   Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cpu
Use FP16 precision: False

05/05/2021 15:40:56 - INFO - __main__ -   Accelerator local main process: True
05/05/2021 15:40:56 - INFO - root -   The wrapper has started
05/05/2021 15:40:56 - INFO - root -   KeyPH: you can choose one or multiple datasets to load all in one.
05/05/2021 15:40:56 - INFO - root -   [INFO-START] Multi dataset read
05/05/2021 15:40:56 - INFO - root -   [INFO-END] Multi dataset read


loaded result from cache for function json_keyphrase_read.<locals>._json_keyphrase_read    
[{'test': Dataset({
    features: ['name', 'title', 'abstract', 'fulltext', 'keywords'],
    num_rows: 500
}), 'valid': Dataset({
    features: ['name', 'title', 'abstract', 'fulltext', 'keywords'],
    num_rows: 1500
}), 'train': Dataset({
    features: ['__index_level_0__'],
    num_rows: 0
})}]


ArrowInvalid: Must pass at least one table